In [28]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #for plotting the 3-D plot.
from scipy import interpolate
from labels_generator import Label_generator
from data_gen_utils import *

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
pts = generate_traj(show=True)


In [36]:

import random
import re
import json

%matplotlib qt

MAX_NUM_OBJS = 6
MIN_NUM_OBJS = 2


margin = 0.2
pts = generate_traj(n_wp = 40, N=100,n_int= 10,show=False, margin=margin)

t_ag = Text_augmentor()

num_objs = 5


all_obj_names = ["table","chair","tree", "rock", "wall", "bycicle", "car", "computer", "cellphone", "TV"]

obj_names = random.sample(all_obj_names, k = num_objs)
obj_pt = np.random.random([num_objs,3])*(1-margin)+margin/2
objs  = {}
for x,y,z,name in zip(obj_pt[:,0],obj_pt[:,1],obj_pt[:,2],obj_names):
    objs[name] = {"value":{"obj_p":[x,y,z]}}



lg = Label_generator(objs)
# change_type = ['dist']
change_type = ['dist','speed', 'cartesian']

lg.generate_labels(change_type)

print(len(lg.labels))
# for k, v in lg.labels:
#     print(k)

for i,(text, map_cost_f) in enumerate(lg.sample_labels(1)):
# for i, (text, map_cost_f) in enumerate(l_set):

    map_cost_f_list = [map_cost_f]
    num_interactions = 2

    for j in range(num_interactions-1):
        new_text, new_map_cost_f = lg.sample_labels(1)[0]
        map_cost_f_list.append(new_map_cost_f)
        text+=" and "+new_text

    print("ORIGINAL:", text)
    pts_new = apply_force(pts,map_cost_f_list)[0][0]

    text_aug = t_ag.augment(text,10,10)
    print("AUGMENTED:")
    for ti in text_aug:
        print(ti)
    plot_samples(text,pts,[pts_new],objs=objs, color_traj  =True)

    print("DONE")



2974
ORIGINAL: drive a little slower when passing nearby the computer and walk a bit closer to the cellphone
AUGMENTED:
Walk a bit closer to the cellphone when passing by the computer.
Walk a bit closer to the cellphone when passing by a computer.
Walk a little closer to the cellphone when passing by the computer.
Walk a bit closer to the cellphone when passing the computer.
Walk a bit closer to the cellphone when passing by the computer and drive a little slower.
Walk a bit closer to the cellphone when passing the computer and drive a little slower.
Walk a bit closer to the cellphone when passing near the computer.
Walk a bit closer to the cellphone when passing a computer.
Walk a bit closer to the cellphone when you pass the computer.
Walk a bit closer to the cellphone and drive a little slower when passing the computer.
DONE


In [51]:
dg = data_generator({'dist':1,'speed':1, 'cartesian':1}, obj_lib_file= "/home/mirmi/Arthur/NL_trajectory_reshaper/imagenet1000_clsidx_to_labels.txt")
data = dg.generate(2,4)
print(len(data))
# dg.next()
# dg.next()


8


In [65]:


i=4
pts = data[i]["input_traj"]
pts_new = data[i]["output_traj"]
text = data[i]["text"]
obj_names = data[i]["obj_names"]
obj_pt = data[i]["obj_poses"]
obj_classes = data[i]["obj_classes"]
obj = data[i]["obj_in_text"]
lg_ct = data[i]["change_type"]
mi = data[i]["map_id"]

objs  = {}
for x,y,z,name in zip(obj_pt[:,0],obj_pt[:,1],obj_pt[:,2],obj_names):
    objs[name] = {"value":{"obj_p":[x,y,z]}}

images_path = "/home/mirmi/Arthur/dataset/"
objs_img_base_paths = [images_path+c+"/"+n for c,n in zip(obj_classes,obj_names)]


import os
from PIL import Image

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

obj_img_paths = [im_path+"/"+random.choice(os.listdir(im_path)) for im_path in objs_img_base_paths]
objs_images = [load_image(im) for im in obj_img_paths] 

print()
plot_samples(text,pts,[pts_new],images=objs_images,objs=objs, color_traj  =True)



: 

In [24]:
ct = {'dist':0,'speed':0, 'cartesian':0}
for d in data:
    ct[d["change_type"]]=ct[d["change_type"]]+1
print(ct)

{'dist': 292, 'speed': 368, 'cartesian': 340}


In [40]:

text ="Pass a bit slower while passing close to the table."
print("ORIGINAL:", text)

text_aug = t_ag.augment(text,3,10)
print("AUGMENTED:")
for ti in text_aug:
    print(ti)

ORIGINAL: Pass a bit slower while passing close to the table.
AUGMENTED:
While passing close to the table, pass a bit slower.
While passing close to the table pass a bit slower.
It's a good idea to pass a bit slower.


In [37]:
for d in data:
    print(d["text"])

stay on the upper part
stay on the front
go to the top
staydown
drive a bit faster while passing near the banana
walk a little slower when passing next to the worm fence
drive a bit faster while passing nearby the worm fence
drive a bit slower when passing in the surrounding of the worm fence
stay closer to the goose
walk a bit further away from the goose
drive a little closer to the blower
pass a bit closer to the blower
drive very closer to the spider's web
keep a bigger distance from the hermit crab
pass a little further away from the Dungeness crab
stay further away from the jigsaw puzzle
go to the front
stay on the bottom
stay on the bottom part
go to the top
drive very slower when passing near the chest
drive a bit slower when passing in the proximity of the honeycomb
increase the speed when passing next to the pinwheel
increase the speed while passing in the proximity of the keeshond
go to the front
go to the bottom part
right
bottom
walk very slower in the surrounding of the Pa

In [ ]:
model_name = 'tuner007/pegasus_paraphrase' 
torch_device = 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def get_response(input_text: str, num_return_sequences: int, num_beams: int) -> List[str]:
      batch = tokenizer.prepare_seq2seq_batch([input_text], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
      translated = model.generate(**batch, max_length = 60, num_beams = num_beams, num_return_sequences = num_return_sequences, temperature = 1.5)
      tgt_text = tokenizer.batch_decode(translated, skip_special_tokens = True)
      return tgt_text

context = "Which course should I take to get started in data science?"
num_return_sequences = 10
num_beams = 10
get_response(context, num_return_sequences, num_beams)

['Which data science course should I take?',
 'Which data science course should I take first?',
 'Should I take a data science course?',
 'Which data science class should I take?',
 'Which data science course should I attend?',
 'I want to get started in data science.',
 'Which data science course should I enroll in?',
 'Which data science course is right for me?',
 'Which data science course is best for me?',
 'Which course should I take to get started?']